In [ ]:
import quandl
import pandas as pd
import numpy as np
from datetime import datetime 
import pyodbc 
from sqlalchemy import create_engine

# ustanawiam połączenie z bazą
SERVER = 'bigmacindex.database.windows.net'
DATABASE = 'bigmacindex'
DRIVER = 'ODBC Driver 17 for SQL Server'

# docelowo folder z credentialami nie byłby ogólno dostępny, ale chciałem żeby skrypt można było uruchomić 
with open('creds/data_writer_creds.txt','r') as f:
    creds = [line.strip() for line in f.readlines()]

USERNAME = creds[0] 
PASSWORD = creds[1] 


DATABASE_CONNECTION = f'mssql://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER}'

engine = create_engine(DATABASE_CONNECTION)
connection = engine.connect()

# pobieram tabelę geographies z aktualnymi rekordami
geographies = pd.read_sql("select * from geographies where active = 1", connection)

# otwieram aktualną listę krajów i sortuję w taki sam sposób jak w bazie
economist_country_codes = pd.read_csv('data/economist_country_codes_actual.csv',sep=';|\|',engine='python')
economist_country_codes = economist_country_codes.sort_values('COUNTRY',ignore_index=True) 

# za pomocą outer joina szukam regionów, które się zmieniły
geographies = geographies.merge(economist_country_codes, left_on = ['country_code','region'], 
                                right_on = ['CODE','REGION'], how = 'outer')
geographies_new = geographies[geographies['geography_id'].isna()]

geographies_old = geographies[geographies['COUNTRY'].isna()]

# nadaje nowe wartości polu geography_id dla nowych rekordów
geographies_new = geographies_new.reset_index()
geographies_new['geography_id']=geographies_new.index+1+max(geographies['geography_id'])

# updatuję geographies tabelę zmieniając pola valid_to oraz active dla starych rekordów oraz dodaje nowe rekordy
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)
cursor = cnxn.cursor()

today = datetime.today().strftime('%Y-%m-%d')
end = '9999-01-01'


for index, row in geographies_old.iterrows():
    cursor.execute("UPDATE geographies SET active=0, valid_to=? where geography_id=?", today, int(row.geography_id))
    

for index, row in geographies_new.iterrows():
    cursor.execute("""INSERT INTO geographies (geography_id, country_code, country_name, region, valid_from, valid_to, active)
    values (?,?,?,?,?,?,1)""", int(row.geography_id), row.CODE, row.COUNTRY, row.REGION, today, end)

cnxn.commit()
connection.close()
cursor.close()
cnxn.close()